# **Setup**
 
Reset the Python environment to clear it of any previously loaded variables, functions, or libraries. Then, import the libraries needed to complete the code Professor Melnikov presented in the video.

In [ ]:
%reset -f
from IPython.core.interactiveshell import InteractiveShell as IS
IS.ast_node_interactivity = "all"    # allows multiple outputs from a cell
import numpy as np, seaborn as sns, matplotlib.pyplot as plt

<hr style="border-top: 2px solid #606366; background: transparent;">

# **Review**

## Matrix Factorization (Decomposition)

Many advanced algorithms, such as Singular Value Decomposition (SVD) used in topic modeling, use matrix decomposition to find "simple" patterns in observed matrix values. Before you move further into these advanced algorithms, you will learn about matrix decompositions &mdash; what they are, when they are applicable, and what to expect as a result.

In this notebook you will perform matrix factorization (a.k.a. decomposition) on popular special matrices. 
 
You will start by creating a function that takes matrices `A`, `B`, and `C` as arguments and plots them as heatmaps for visual convenience.

In [ ]:
def PlotMatMult(A, B, C, figSize=[10,2], nPrecision=2, LsLab=['A', 'B', 'C']) -> None:
    '''Plots heatmap subplots with 3 matrices in a row
    Inputs:
        A,B,C: numpy 2D arrays (i.e. matrices)
        figSize: width and height dimensions of the figure to be plotted
        nPrecision: number of digits to display in annotations
        LsLab: list of labels for the A,B,C matrices  '''
    plt.rcParams['figure.figsize'] = figSize
    fig, (axA, axB, axC) = plt.subplots(1, 3)
    fig.suptitle(f'{LsLab[0]} ⋅ {LsLab[1]} = {LsLab[2]}', fontsize=15)

    def Heatmap(Arr:'numpy 2D array', ax:'axis object', sLabel='') -> None:
        '''Helper function to plot a single matrix as a heatmap
        Inputs:
            Arr: matrix to plot as a heatmap, i.e. a 2D array
            ax: matplotlib axis object, i.e. a reference to a subplot
            sLabel: label for the plot        '''
        ax1 = sns.heatmap(Arr, annot=Arr.round(nPrecision), cbar=False, cmap='coolwarm', fmt='',
                          annot_kws={"fontsize":15}, ax=ax, xticklabels=False, yticklabels=False);
        ax1.tick_params(left=False, bottom=False);
        ax.set(xlabel=sLabel + ' ' + str(Arr.shape));
  
    Heatmap(A, axA, LsLab[0])
    Heatmap(B, axB, LsLab[1])
    Heatmap(C, axC, LsLab[2])
    fig.tight_layout(rect=[0, 0.03, 1, 0.95])
    plt.show()

<span style="color:black">Create two matrices, `A` and `B`, and use the previous function to plot these with their matrix product (matrix `C`). Think of this equation as:
1. Product of $A,B$ to produce $C$, or
1. Decomposition (i.e., factorization) of the given matrix $C$ into $A,B$
 
<span style="color:black">This is similar to factorization of a real number. For example, you can factorize $1=a\cdot 1/a$ for any $a\ne0$. There are infinitely many $a$ values. In general, $C$ can also be factored into infinitely many matrices. The dimensionality of $A,B$ can change as well, as long as the inner dimensions are the same and outer dimensions match that of the shape of $C$.

In [ ]:
A = np.array(
    [[1, 0, 1],
     [0, 1, 0]])
B = np.array(
    [[1, 0, 1, 0],
     [0, 1, 0, 1],
     [1, 0, 1, 0]])

PlotMatMult(A, B, A @ B)

## Matrix Multiplication


<span style="color:black">Unlike real numbers, matrix multiplication can produce the same output from an infinite combination of factors.
<div style="margin-top: 20px; margin-bottom: 20px;">
<details style="border: 2px solid #ddd; margin-bottom: -2px;">
    <summary style="padding: 12px 15px; cursor: pointer; background-color: #eee;">
        <div id="button" style="padding: 0px;">
            <font color=#B31B1B>▶ </font> 
            <b> Example:</b> Infinite Factors in Matrix Multiplication
        </div>
    </summary>
    <div id="button_info" style="padding:10px"> 

Divide `A` by 2 and multiply `B` by 0.5 to derive matrices `A1` and `B1`. These matrices are different, but their product is still the same matrix `C`. The following statements show that $AB$ is equivalent to $A_1\cdot B_1$:
 
$$C=AB=A\cdot 1\cdot B=A\cdot (2 \cdot 0.5)\cdot B=(A\cdot 2)\cdot(0.5\cdot B)=A_1\cdot B_1$$
 
Since no change occurs when a matrix is multiplied by scalar 1, the following is true: $AB=A\cdot 1\cdot B$. Multiplication by 1 can be rewritten as $2\cdot0.5$, and each of these scalars can be associated with a matrix to obtain $A\cdot 1\cdot B=(A\cdot 2)\cdot(0.5\cdot B)=A_1\cdot B_1$. Note that by construction, $A\ne A_1$ and $B\ne B_1$. 

You can have infinitely many factors of 1. For example, you can represent 1 as $3\cdot\frac{1}{3}$, $-1\cdot-1$, $10.5\cdot\frac{1}{10.5}$ and so on. In general, $1=v\cdot\frac{1}{v}$ for any non-zero real value $v$. Since there are infinitely many real numbers, there are infinitely many matrix factors, which will produce the same matrix `C` above. 
    </div>
</details>
</div>

<span style="color:black">Verify this statement by comparing the plot from multiplying `A1` and `B1` with the above plot.

In [ ]:
A1, B1 = 0.5 * A,   2 * B
PlotMatMult(A1, B1, A1 @ B1, LsLab=['A1', 'B1', 'C1'])

<span style="color:black">Here is one more version of factor matrices `A2` and `B2`, which gives the same `C` as a product.

In [ ]:
A2, B2 = -1 * A,   -1 * B
PlotMatMult(A2, B2, A2 @ B2, LsLab=['A2', 'B2', 'C2'])

<span style="color:black">You can rescale matrices to derive the same product. But, can you find two different matrices (unrelated via scaling) to derive the same product? The answer is yes, and there is no reason to look for an overly complicated example. The two matrices below give us the zero matrix (full of zero values) if multiplied as $XY$ or $YX$. You should verify this multiplication with pencil and paper.

In [ ]:
X = np.array(  # Example of different factors yield the same product
    [[1, 0], 
     [0, 0]])
Y = np.array(
    [[0, 0], 
     [0, 1]])

PlotMatMult(X, Y, X @ Y, figSize=[10,1.5], LsLab=['X', 'Y', 'X@Y'])
PlotMatMult(Y, X, Y @ X, figSize=[10,1.5], LsLab=['Y', 'X', 'Y@X'])

## Special Matrices


<span style="color:black">You may already know many numbers with special properties, such as $a * 0$, an additive identity, which does not change another summand, i.e. $0+a=a+0=a$ for any number $a * 1$, a multiplicative identity, which does not change another multiplicative factor, i.e., $1\cdot a=a\cdot1=a$ for any number $a$.
 
<span style="color:black">There are matrices with similar properties. However, the world of special matrices is far more diverse.
<div style="margin-top: 20px; margin-bottom: 20px;">
<details style="border: 2px solid #ddd; margin-bottom: -2px;">
    <summary style="padding: 12px 15px; cursor: pointer; background-color: #eee;">
        <div id="button" style="padding: 0px;">
            <font color=#B31B1B>▶ </font> 
            <b> More About:</b> Special Matrix Types
        </div>
    </summary>
    <div id="button_info" style="padding:10px"> 

1. A **zero** matrix is an additive identity, assuming matching dimensions. In this matrix all values are zeros but dimensions can vary.
1. An **identity** matrix is a multiplicative identity, assuming matching inner dimensions. This is a square matrix of varying dimensions with ones on diagonal and zeros otherwise.
1. A matrix of ones, where all values are ones.
1. A **diagonal** line or elements in a matrix includes all elements with matching row/column indices. So, in a matrix $[a_{ij}]$, $a_{ii}$ are diagonal elements, and other elements are off-diagonal.
1. A **symmetric** matrix where all values are the same about the diagonal line or elements.
    1. A symmetric matrix must be square, i.e., have the same number of rows and columns.
    1. A symmetric matrix has many amazing properties that ease operations on it, such as factorization into two matrices.
1. A **diagonal** matrix is a matrix with zeros off-diagonal &mdash; above and below the diagonal values. Diagonal values are those with equal indices, but the matrix itself does not need to be square. 
    1. Ex. Let $A:=[a_{ij}]_{2\times4}$, i.e., a matrix with two rows and four columns and elements $a_{ij}$. Then $A$ is diagonal, if $a_{ij}=0,\forall i\ne j$, i.e., for all values with non-equal indices being zeros. The zero matrix is also diagonal by this definition.
    </div>
</details>
</div>


In [ ]:
print('Zero matrix, np.zeros((2,4)):\n', np.zeros((2,4), dtype=int))  # similar functionality to a number 0
print('Identity matrix, np.identity((2,4)):\n', np.eye(3, dtype=int)) # similar functionality to a number 1
print('Matix of ones, np.ones((2,4)):\n', np.ones((2,4), dtype=int))    
print('Symmetric matrix, B.T @ B:\n', B.T @ B)                        # symmetric about its diagonal line. It's always square
print('Diagonal matrix, np.diag([1,2,3]):\n', np.diag([1,2,3]))       # diagonal matrices are symmetric

## Orthogonal and Orthonormal Vectors

<span style="color:black">**Orthogonal** (or perpendicular) vectors are defined as those that give a zero in a dot product. So $a\top:=[1,0]$ and $b\top:=[0,-1]$ are orthogonal, since their dot product is zero. Recall that mathematically you always represent vectors vertically, so you use a transposition to indicate that $a$ is a vertical vector, not a horizontal one you display. The (Euclidean) length of $a$ is 1, but length of $b$ is 7. If you scale $b$ as $\hat b:=b/\text{length}(b)=[-1,0]$, you derive another unit-length vector. Now, these two vectors are not just orthogonal, but **orthonormal**. 
 
Now, a matrix with orthogonal vectors as columns is orthogonal. Likewise, a matrix with orthonormal vectors as columns is orthonormal. If you multiply an orthogonal matrix by its own transpose, you derive a diagonal matrix (with possibly non-zero values on diagonal). If you multiply an orthonormal matrix by its own transpose, you derive an identity matrix. This is because diagonal elements are dot products of each column with itself, which gives a value 1. Confirm this by computing the dot product $a\bullet a$.
 
Define two orthonormal vectors, $x$ and $y$, and package them as columns of matrix $Z$ using column concatenation operation in NumPy, [`np.c_`](https://numpy.org/doc/stable/reference/generated/numpy.c_.html).

In [ ]:
x = np.array([1,0])   # unit-length vector along x-axis in 2D Cartesian coordinates
y = np.array([0,-1])  # unit-length vector along y-axis in 2D Cartesian coordinates
Z = np.c_[x,y]        # x and y are columns in matrix Z
PlotMatMult(Z, Z.T, Z @ Z.T, figSize=[10,1.5], LsLab=['Z', 'Z.T', 'Z @ Z.T'])

<span style="color:black">There are infinitely many orthogonal matrices, i.e., matrices that have orthogonal column vectors. Use [`ortho_group.rvs`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.ortho_group.html) to generate a random orthogonal matrix, $Z$ and then verify that it produces an identity matrix (up to a tiny numerical error) by multiplying $Z$ with its own transpose.

In [ ]:
from scipy.stats import ortho_group
np.set_printoptions(suppress=True)

Z = ortho_group.rvs(dim=3, random_state=0)   # creates some orthogonal matrix
PlotMatMult(Z, Z.T, Z @ Z.T, figSize=[10,1.5], LsLab=['Z', 'Z.T', 'Z @ Z.T'])

<hr style="border-top: 2px solid #606366; background: transparent;">

# **Optional Practice**
Now, equipped with these concepts and tools, you will tackle a few related tasks.

As you work through these tasks, check your answers by running your code in the *#check solution here* cell, to see if you’ve gotten the correct result. If you get stuck on a task, click the See **solution** drop-down to view the answer.

# Task 1
 
Define $Z:=[0]_{5\times 9}$, i.e. a $5\times 9$ zero matrix. Then define $U:=[1]$ as the matrix of ones of the appropriate size. Then define $F:=5U$ as 5 times $U$. Then define $I$ as an identity matrix of the appropriate size. Finally, compute matrix $A:=((Z + F) \odot U) \cdot I$, where $\odot$ is the Hadamard (i.e., element-wise) product and $\cdot$ is the usual matrix product. Is $A$ symmetric or diagonal?
 
P.S. $\odot, \oslash, \otimes, \oplus, \ominus$ are often used in textbooks and publications to denote element-wise operations between vectors or between matrices.

<b>Hint:</b> Use NumPy operations from the examples above to create these matrices. In order to identify the "appropriate" dimensions of each matrix, write down the final equation and work backwards noting that addition requires both matrices to have the same size and matrix multiplication requires inner dimensions to match. Also, check precise definitions of symmetric, diagonal and identity matrices above. Do any properties fail these definitions?

In [ ]:
# check solution here


<font color=#606366>
    <details><summary><font color=#B31B1B>▶ </font>See <b>solution</b>.</summary>
$A$ is a matrix of element values 5. Since it is not square, it cannot be symmetric. Diagonal matrices do not need to be square, but they must have zeros above an below diagonal values.
            <pre>
Z = np.zeros((5, 9), dtype=int)
U = np.ones(Z.shape, dtype=int)
F = 5 * U
I = np.eye(Z.shape[1])
A = ((Z + F) * U) @ I
A
</pre>
</details> 
</font>
<hr>

# Task 2
 
Define $B$ as a top left submatrix of $I$ of the appropriate size to compute $C:=U+9B$. Is $C$ symmetric, diagonal or identity?

Note: top left $n\times p$ submatrix of the matrix $X$ is the block (or submatrix) of contiguous $n$ leftmost rows and $n$ topmost columns.

<b>Hint:</b> Just like above deduce the shape of all matrices from the final equation. Also, check precise definitions of symmetric, diagonal and identity matrices above. Do any properties fail these definitions?

In [ ]:
# check solution here


<font color=#606366>
    <details><summary><font color=#B31B1B>▶ </font>See <b>solution</b>.</summary>
$C$ is not symmetric, since it's not an even square, and not diagonal, since off-diagonal values are not zeros. For the same reasons, it is not an identity matrix.
            <pre>
B = I[:U.shape[0], :U.shape[1]]  # ensures equal number of rows/columns between U and I2
C = U + 9*B
C
</pre>
</details> 
</font>
<hr>

# Task 3
 
Compute $D_1:=C^\top C$ and $D_2:=C C^\top$ as matrix multiplications. Are any of the resulting matrices symmetric, diagonal, or identity?

<b>Hint:</b> Look up the transpose operation above for NumPy matrices. Also, check precise definitions of symmetric, diagonal and identity matrices above. Do any properties fail these definitions?

In [ ]:
# check solution here

<font color=#606366>
    <details><summary><font color=#B31B1B>▶ </font>See <b>solution</b>.</summary>
$D_1,D_2$ are symmetric because they are square and have matching elements in positions symmetric about the diagonal line. However, these are not diagonal matrices and not identity matrices because they fail definitions of having zeros above and below the diagonal line.
            <pre>
D1 = C.T @ C
D2 = C @ C.T
D1
D2
</pre>
</details> 
</font>
<hr>

# Task 4
 
Define $U_2$ as a top left submatrix of $U$ of the appropriate size. Then compute $Y:=(D_2-27U_2)/81$. Is $Y$ symmetric, diagonal, or identity?

<b>Hint:</b> Again, work backwards from the final equation to determine the appropriate dimensions of each matrix.

In [ ]:
# check solution here


<font color=#606366>
    <details><summary><font color=#B31B1B>▶ </font>See <b>solution</b>.</summary>
$Y$ is symmetric, diagonal and identity because it satisfies each of these definitions (see above). 
            <pre>
U2 = U[:D2.shape[0], :D2.shape[1]]
(D2 - 27*U2)/81
</pre>
</details> 
</font>
<hr>